# Import

In [3]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

# Reading Session Keys

In [4]:
session_key_list = pd.read_csv('session_keys.csv')

In [8]:
for i in session_key_list['session_key']:
    response = urlopen(f'https://api.openf1.org/v1/laps?session_key={i}')
    data = json.loads(response.read().decode('utf-8'))
    if i == 7953:
        df_laps = pd.DataFrame(data)
    else:
        hold = pd.DataFrame(data)
        df_laps = pd.concat([df_laps,hold])
        time.sleep(0.5)
    print(f'Session {i} extracted.')
print('Done')

Session 7953 extracted.
Session 7779 extracted.
Session 7787 extracted.
Session 9070 extracted.
Session 9078 extracted.
Session 9094 extracted.
Session 9102 extracted.
Session 9110 extracted.
Session 9118 extracted.
Session 9126 extracted.
Session 9133 extracted.
Session 9141 extracted.
Session 9149 extracted.
Session 9157 extracted.
Session 9165 extracted.
Session 9173 extracted.
Session 9221 extracted.
Session 9213 extracted.
Session 9181 extracted.
Session 9205 extracted.
Session 9189 extracted.
Session 9197 extracted.
Session 9472 extracted.
Session 9480 extracted.
Session 9488 extracted.
Session 9496 extracted.
Session 9673 extracted.
Session 9507 extracted.
Session 9515 extracted.
Session 9523 extracted.
Session 9531 extracted.
Session 9539 extracted.
Session 9550 extracted.
Session 9558 extracted.
Session 9566 extracted.
Session 9574 extracted.
Session 9582 extracted.
Session 9590 extracted.
Session 9598 extracted.
Session 9606 extracted.
Session 9617 extracted.
Session 9625 ext

In [9]:
df_laps

,meeting_key,session_key,driver_number,lap_number,date_start,duration_sector_1,duration_sector_2,duration_sector_3,i1_speed,i2_speed,is_pit_out_lap,lap_duration,segments_sector_1,segments_sector_2,segments_sector_3,st_speed
0,1141,7953,1,1,None,NaN,42.414,23.842,232.0,231.0,False,NaN,"[None, 2048, 2049, 2049, 2051, 2051, 2049, 204...","[2051, 2049, 2051, 2049, 2051, 2049, 2049, 204...","[2051, 2049, 2049, 2049, 2051, 2051]",252.0
1,1141,7953,11,1,None,NaN,43.216,24.069,230.0,237.0,False,NaN,"[None, 2048, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049]",252.0
2,1141,7953,16,1,None,NaN,42.549,24.122,226.0,250.0,False,NaN,"[None, 2048, 2049, 2051, 2049, 2049, 2049, 204...","[2049, 2051, 2049, 2049, 2049, 2049, 2049, 205...","[2049, 2049, 2049, 2049, 2049, 2049]",255.0
3,1141,7953,31,1,None,NaN,45.213,24.698,233.0,238.0,False,NaN,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]",254.0
4,1141,7953,27,1,None,NaN,45.924,24.741,220.0,239.0,False,NaN,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 0]",249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,1252,9662,10,58,2024-12-08T14:29:50.414000+00:00,17.995,38.878,33.992,289.0,285.0,False,90.865,"[None, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...",302.0
1033,1252,9662,27,58,2024-12-08T14:29:54.299000+00:00,18.096,38.630,33.314,292.0,299.0,False,90.040,"[2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...",310.0
1034,1252,9662,20,57,2024-12-08T14:30:00.785000+00:00,17.257,37.109,31.271,295.0,300.0,False,85.637,"[2051, 2051, 2051, 2051, 2049]","[2051, 2048, 2051, 2048, 2048, 2051, 2049, 204...","[2051, 2049, 2049, 2051, 2051, 2051, 2051, 204...",312.0
1035,1252,9662,14,58,2024-12-08T14:30:02.564000+00:00,17.619,38.418,32.584,291.0,301.0,False,88.621,"[None, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...",300.0


In [10]:
config = dotenv_values()

api_key = config['MeteostatAPI-Key']
pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

python-dotenv could not parse statement starting at line 8
python-dotenv could not parse statement starting at line 9


In [11]:
df_laps.columns

Index(['meeting_key', 'session_key', 'driver_number', 'lap_number',
       'date_start', 'duration_sector_1', 'duration_sector_2',
       'duration_sector_3', 'i1_speed', 'i2_speed', 'is_pit_out_lap',
       'lap_duration', 'segments_sector_1', 'segments_sector_2',
       'segments_sector_3', 'st_speed'],
      dtype='object')

In [12]:
dtype_dict = {
    'meeting_key': types.Integer,
    'session_key': types.Integer,
    'driver_number': types.Integer,
    'lap_number': types.Integer,
    'date_start': types.DateTime,
    'duration_sector_1': types.Float,
    'duration_sector_2': types.Float,
    'duration_sector_3': types.Float,
    'i1_speed': types.Float,
    'i2_speed': types.Float,
    'is_pit_out_lap': types.Boolean,
    'lap_duration': types.Float,
    'segments_sector_1': types.String,  # likely encoded string or JSON-like
    'segments_sector_2': types.String,
    'segments_sector_3': types.String,
    'st_speed': types.Float
}

In [13]:
df_laps.to_sql(name = 'raw_laps', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

108